# BQMLに新たに追加されたTRANSFORM句で、予測時のモデルの前処理を自動的に行う

# これなに?

BQMLに新たに築かされた`TRANSFORM`句についての解説記事です。2019/12/2時点で、まだ日本語の公式ドキュメントが存在しないことから、記事にしようと思いました。なお、現時点ではまだこの機能は`Beta`です。[英語の公式ドキュメント](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create)は存在するので、興味があれば、こちらも参考することをお勧めします。

# TRANSFORM句とは?

**行いたい前処理をモデル構築時に定義し、予測、評価時に自動的に実行する**ためにしようするSQLの句(clause)です。  
これにより、BQMLで作成するアルゴリズムとそれに伴う前処理を一体化させ、モデルを構築することができます。

# 具体例

今回は例として、`bigquery-public-data.samples.natality` 配下にある、新生児のデータを使用し、出産時の体重を目的変数とするモデルを構築してみたいと思います

In [24]:
% % bigquery --project $PROJECT
SELECT
  weight_pounds,
  -- 目的変数
  is_male,
  plurality,
  --一回の妊娠で生まれた子供の数
  gestation_weeks,
  -- 妊娠期間
  alcohol_use,
  -- 母親が飲酒してたか
  cigarette_use -- 母親がタバコを吸っていたか
FROM
  `bigquery-public-data.samples.natality`
LIMIT
  5

,weight_pounds,is_male,plurality,gestation_weeks,alcohol_use,cigarette_use
0,7.625790,True,NaN,38,None,None
1,7.438397,False,1.0,38,None,None
2,8.437091,False,1.0,41,None,None
3,7.374463,True,1.0,99,None,None
4,5.813590,False,1.0,99,None,None


BQMLの場合、そのままデータを入力しても基本的な前処理は自動で行ってくれますが、自分で特徴量を作成したほうがより良い精度が期待できます。
そこで、以下のような特徴量を作成します。
- 多胎児で生まれたか否か
- 妊娠期間が37-42週に当てはまっているかどうか
- (母親が)アルコールを摂取していたかと、タバコを吸っていたかの交差特徴量

In [40]:
%%bigquery --project $PROJECT
SELECT
  weight_pounds,
  is_male,
 
  IF(plurality > 1, 1, 0) AS is_plurality,
  ML.BUCKETIZE(gestation_weeks, [37, 42]) AS bined_gestation_weeks,
  ML.FEATURE_CROSS(
    STRUCT(
      CAST(alcohol_use AS STRING) AS alcohol_use,
      CAST(cigarette_use AS STRING) AS cigarette_use
    )
  ) AS alcohol_cigarette_use
FROM
  `bigquery-public-data.samples.natality`
WHERE alcohol_use is not null
LIMIT
  5

,weight_pounds,is_male,plurality,is_plurality,bined_gestation_weeks,gestation_weeks,alcohol_cigarette_use
0,8.911085,False,1,0,bin_2,40,{'alcohol_use_cigarette_use': None}
1,8.146081,True,1,0,bin_2,39,{'alcohol_use_cigarette_use': None}
2,5.485101,True,1,0,bin_1,35,{'alcohol_use_cigarette_use': None}
3,7.098885,False,1,0,bin_2,37,{'alcohol_use_cigarette_use': None}
4,8.463546,True,1,0,bin_3,43,{'alcohol_use_cigarette_use': None}


上記の関数については、[公式ドキュメント](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions?hl=ja)か、[以前の記事](https://qiita.com/Hase8388/items/5fcc9f056d44105d186e)を参照してください。

さて、問題は、ここからです。上記の処理をした上でモデルに食わせ、学習させる必要がありますが、**そのままだと、予測時に,もう一度同じ前処理を行った上でモデルに入力する必要がありました**これだと、二度同じ処理が発生するともに、うっかり学習時と違う処理をしてしまうことで、面倒なバグを生む温床になりかねません

In [ ]:
-- TRANSFORM句を使わない場合

In [ ]:
## TRANSFORMを使う